In [1]:
# assign directory
import os
BRANDON = '/Users/brandonmarks/Desktop/Research Materials/hierarchical-bayesian-model-validation/'
YASH = '/Users/yashd/Desktop/hierarchical-bayesian-model-validation/'

ROOT_DIR = YASH

RAW_DATA_SUFFIX = "agriVision-RGB-cleaned" 
data_dir = ROOT_DIR + "raw-data/agriVision/full-" + RAW_DATA_SUFFIX # raw-data/agriVision/full-agriVision-RGB-cleaned
# iterate over files in that directory
file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(data_dir)

In [2]:
os.chdir(ROOT_DIR + "dataset-preparation/")
from transform_utilities import *

In [3]:
'''To split large dataset into many batches, only needs to be run once'''
# k = 10000
# target_dir = ROOT_DIR + 'raw-data/agriVision' # Where the batch{i} folders will be created
# directorySplit(folder_dir = data_dir, target_dir = target_dir, name = RAW_DATA_SUFFIX, k = k)
# print(f"{len(file_names)//k} batches created" )

'To split large dataset into many batches, only needs to be run once'

In [4]:
print(''.join([x+"\n" for x in os.listdir("../raw-data/agriVision") if x.__contains__(RAW_DATA_SUFFIX)]))
BATCH_NUM = 3

batch0-agriVision-RGB-cleaned
batch1-agriVision-RGB-cleaned
batch2-agriVision-RGB-cleaned
batch3-agriVision-RGB-cleaned
batch4-agriVision-RGB-cleaned
batch5-agriVision-RGB-cleaned
batch6-agriVision-RGB-cleaned
batch7-agriVision-RGB-cleaned
batch8-agriVision-RGB-cleaned
full-agriVision-RGB-cleaned
toy-agriVision-RGB-cleaned



# Fourier

In [5]:
#Values obtained from plots in agriVisionFourierEDA.ipynb
batch_dir = ROOT_DIR + f'raw-data/agriVision/batch{BATCH_NUM}-agriVision-RGB-cleaned'
splits = getSplits(0.008052940675034493,0.5282291467669505, 1.35)
print(splits)

[0.008052940675034493, 0.010871469911296566, 0.014676484380250364, 0.01981325391333799, 0.02674789278300629, 0.0361096552570585, 0.04874803459702898, 0.06580984670598912, 0.08884329305308532, 0.11993844562166518, 0.161916901589248, 0.21858781714548484, 0.29509355314640456, 0.3983762967476462]


In [6]:
red_fr = convert_to_fourier_basis(batch_dir, "Red", split_list = splits, debug = True)
pd.to_pickle(red_fr, f'../transformed-data/batch{BATCH_NUM}-agriVision-red-fourier.pickle')
red_fr.head()

100%|██████████| 10554/10554 [04:31<00:00, 38.85it/s]


[0.00805294 0.01087147 0.01467648 0.01981325 0.02674789 0.03610966
 0.04874803 0.06580985 0.08884329 0.11993845 0.1619169  0.21858782
 0.29509355 0.3983763 ]


100%|██████████| 14/14 [01:38<00:00,  7.03s/it]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,Red,"(0.0, 0.0078125)",10,"[9870.0, 328.0, 22040.0, 30536.0, 329061.0, 21..."
1,2,Red,"(0.008052940675034493, 0.010517900013934578)",6,"[-383.93857342292654, -59.739185788138364, -13..."
2,3,Red,"(0.011048543456039806, 0.014218964627501012)",11,"[1485.1583926909911, 3.499475970413398, -207.5..."
3,4,Red,"(0.014874556847390447, 0.019628663322501738)",18,"[341.38256644501143, 15.057583098176067, 210.1..."
4,5,Red,"(0.019918044974971814, 0.026565372087373914)",30,"[-78.91082482548087, -12.892306241206327, -3.2..."


In [ ]:
blue_fr = convert_to_fourier_basis(batch_dir, "Blue", split_list = splits, debug = True)
pd.to_pickle(blue_fr, f'../transformed-data/batch{BATCH_NUM}-agriVision-blue-fourier.pickle')
blue_fr.head()

In [ ]:
green_fr = convert_to_fourier_basis(batch_dir, "Green", split_list = splits, debug = True)
pd.to_pickle(red_fr, f'../transformed-data/batch{BATCH_NUM}-agriVision-green-fourier.pickle')
green_fr.head()

In [ ]:
gray_fr = convert_to_fourier_basis(batch_dir, "Gray", split_list = splits, debug = True)
pd.to_pickle(red_fr, f'../transformed-data/batch{BATCH_NUM}-agriVision-gray-fourier.pickle')
gray_fr.head()

In [ ]:
ir_fr = convert_to_fourier_basis(batch_dir, "Infrared", split_list = splits, debug = True)
pd.to_pickle(red_fr, f'../transformed-data/batch{BATCH_NUM}-agriVision-ir-fourier.pickle')
ir_fr.head()

[0.00805294 0.01087147 0.01467648 0.01981325 0.02674789 0.03610966
 0.04874803 0.06580985 0.08884329 0.11993845 0.1619169  0.21858782
 0.29509355 0.3983763 ]


,band,channel,magnitude_endpoints,unique_magnitudes,data
0,1,Infrared,"(0.0, 0.0078125)",10,"[80367.0, 464026.0, 265558.0, 232272.0, 394565..."
1,2,Infrared,"(0.008052940675034493, 0.010517900013934578)",6,"[-671.8820488918707, 2292.7830400318353, -270...."
2,3,Infrared,"(0.011048543456039806, 0.014218964627501012)",11,"[-596.3965045249346, -2458.981803649216, 759.5..."
3,4,Infrared,"(0.014874556847390447, 0.019628663322501738)",18,"[-1869.7239977786285, -2382.1295181624646, 108..."
4,5,Infrared,"(0.019918044974971814, 0.026565372087373914)",30,"[-1665.8794472636714, 359.6725136568671, -538...."


# Wavelet

In [ ]:
red_wv = convert_to_wavelet_basis(batch_dir, "Red", debug=True)
#pd.to_pickle(red_wv, '../transformed-data/toy-agriVision-red-wavelet.pickle')
red_wv.head()

10 layers being used


100%|██████████| 10554/10554 [01:28<00:00, 118.69it/s]


,channel,layer,orientation,data
0,Red,1,L1,"[267.7968750000003, 388.21679687500034, 945.17..."
1,Red,2,D,"[-7.015624999999998, 5.251953124999998, -13.79..."
2,Red,2,H,"[6.765624999999998, -31.357421875000036, 78.32..."
3,Red,2,V,"[-267.5468750000003, -60.439453125000085, -209..."
4,Red,3,D,"[0.0, -8.875000000000002, 0.25000000000000006,..."


In [ ]:
green_wv = convert_to_wavelet_basis(data_dir, "Green")
#pd.to_pickle(green_wv, '../transformed-data/toy-agriVision-green-wavelet.pickle')
green_wv.head()

10 layers being used


,channel,layer,orientation,data
0,Green,1,L1,"[159.79101562500017, 913.0214843750011, 440.91..."
1,Green,2,D,"[-54.748046875000064, 60.771484375000185, 6.72..."
2,Green,2,H,"[64.33007812500007, 52.45507812500003, 16.5664..."
3,Green,2,V,"[-90.8027343750001, 0.33007812499994316, -6.49..."
4,Green,3,D,"[9.527343750000007, 5.507812500000005, -2.4296..."


In [ ]:
blue_wv = convert_to_wavelet_basis(data_dir, "Blue")
#pd.to_pickle(blue_wv, '../transformed-data/toy-agriVision-blue-wavelet.pickle')
blue_wv.head()

10 layers being used


,channel,layer,orientation,data
0,Blue,1,L1,"[190.46875000000023, 903.0976562500011, 474.31..."
1,Blue,2,D,"[-79.18359375000007, 59.93750000000008, -0.888..."
2,Blue,2,H,"[90.8710937500001, 49.05078124999999, -14.1269..."
3,Blue,2,V,"[-121.25781250000013, -5.492187500000057, -8.1..."
4,Blue,3,D,"[8.69140625000001, 37.976562500000036, -2.8476..."


In [ ]:
gray_wv = convert_to_wavelet_basis(data_dir, "Gray")
#pd.to_pickle(gray_wv, '../transformed-data/toy-agriVision-gray-wavelet.pickle')
gray_wv.head()

10 layers being used


,channel,layer,orientation,data
0,Gray,1,L1,"[156.96679687500017, 906.3007812500011, 518.66..."
1,Gray,2,D,"[-54.38085937500006, 60.472656250000114, 4.378..."
2,Gray,2,H,"[63.78320312500007, 51.61328125000003, 8.80468..."
3,Gray,2,V,"[-89.6035156250001, -4.45703125, -11.328124999..."
4,Gray,3,D,"[8.62500000000001, 6.179687500000009, -2.51953..."


In [ ]:
ir_wv = convert_to_wavelet_basis(data_dir, "Infrared")
#pd.to_pickle(red_wv, '../transformed-data/toy-agriVision-red-wavelet.pickle')
ir_wv.head()

10 layers being used


,channel,layer,orientation,data
0,Infrared,1,L1,"[156.96679687500017, 906.3007812500011, 518.66..."
1,Infrared,2,D,"[-54.38085937500006, 60.472656250000114, 4.378..."
2,Infrared,2,H,"[63.78320312500007, 51.61328125000003, 8.80468..."
3,Infrared,2,V,"[-89.6035156250001, -4.45703125, -11.328124999..."
4,Infrared,3,D,"[8.62500000000001, 6.179687500000009, -2.51953..."
